<a href="https://colab.research.google.com/github/AtfastrSlushyMaker/pl-standings-prediction-project/blob/main/notebooks/%20model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# --- PREMIER LEAGUE - MODELING PHASE ---
# This notebook uses the processed dataset to train and evaluate ML models

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path  


In [3]:
from pathlib import Path
candidate_paths = [
    Path('data/processed/processed_premier_league_combined.csv'),  # prefer processed dataset (same dir)
    Path('../data/processed/processed_premier_league_combined.csv'),  # processed (one level up)
    Path('../../data/processed/processed_premier_league_combined.csv'),  # processed (two levels up),
    Path('processed_premier_league_combined.csv')
 ]

csv_path = next((p for p in candidate_paths if p.exists()), None)
if csv_path is None:
    raise FileNotFoundError('Dataset not found in expected locations. '
                        'Checked: ' + ', '.join(str(p) for p in candidate_paths))

print(f'Reading: {csv_path}')
df = pd.read_csv(csv_path)
print(df.shape)
df.head()

Reading: ../data/processed/processed_premier_league_combined.csv
(9500, 23)


,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,...,HC,AC,HY,AY,HR,AR,FTR_encoded,Season_encoded,HomeTeam_le,AwayTeam_le
0,2000-01,2000-08-19,Charlton,Man City,4,0,H,17.0,8.0,14.0,...,6.0,6.0,1.0,2.0,0.0,0.0,2,0,12,26
1,2000-01,2000-08-19,Chelsea,West Ham,4,2,H,17.0,12.0,10.0,...,7.0,7.0,1.0,2.0,0.0,0.0,2,0,13,43
2,2000-01,2000-08-19,Coventry,Middlesbrough,1,3,A,6.0,16.0,3.0,...,8.0,4.0,5.0,3.0,1.0,0.0,0,0,14,28
3,2000-01,2000-08-19,Derby,Southampton,2,2,D,6.0,13.0,4.0,...,5.0,8.0,1.0,1.0,0.0,0.0,1,0,16,36
4,2000-01,2000-08-19,Leeds,Everton,2,0,H,17.0,12.0,8.0,...,6.0,4.0,1.0,3.0,0.0,0.0,2,0,22,17


## 🔥 Random Forest: training, cross-validation, and evaluation
This section trains a RandomForest classifier, performs comprehensive cross-validated hyperparameter search, and reports performance metrics (Accuracy, Precision/Recall/F1, ROC/AUC, Confusion Matrix).

In [4]:
# Setup features/target and train/test split
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize

# Define features based on processed dataset docs
all_candidate_features = [
    'Season_encoded','HomeTeam_le','AwayTeam_le',
    'FTHG','FTAG','HS','AS','HST','AST',
    'HF','AF','HC','AC','HY','AY','HR','AR'
 ]

missing = [c for c in all_candidate_features if c not in df.columns]
if missing:
    print('Warning: missing expected features ->', missing)
features = [c for c in all_candidate_features if c in df.columns]

X = df[features].copy()
y = df['FTR_encoded'].astype(int).copy()

# Stratified split to preserve class balance
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42)

print('X shape:', X.shape, '| Train:', X_train.shape, '| Test:', X_test.shape)
print('Classes distribution (train):', np.bincount(y_train))
print('Classes distribution (test):', np.bincount(y_test))

X shape: (9500, 17) | Train: (7600, 17) | Test: (1900, 17)
Classes distribution (train): [2242 1878 3480]
Classes distribution (test): [561 469 870]


In [ ]:
# Random Forest with cross-validated hyperparameter search
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import roc_auc_score, make_scorer

# Parameter grid for comprehensive search
param_grid = {
    'n_estimators': [100, 300, 500],
    'max_depth': [None, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True],
    'class_weight': [None, 'balanced_subsample']
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Fixed: removed 'needs_proba' parameter (deprecated in sklearn 1.6+)
# make_scorer automatically detects if probabilities are needed
auc_scorer = make_scorer(roc_auc_score, multi_class='ovr', average='weighted')

base_rf = RandomForestClassifier(random_state=42, n_jobs=-1)
grid = GridSearchCV(
    estimator=base_rf,
    param_grid=param_grid,
    scoring=auc_scorer,
    cv=cv,
    n_jobs=-1,
    verbose=2,
    return_train_score=True
)

print(f'Starting GridSearchCV with {len(param_grid["n_estimators"]) * len(param_grid["max_depth"]) * len(param_grid["min_samples_split"]) * len(param_grid["min_samples_leaf"]) * len(param_grid["max_features"]) * len(param_grid["class_weight"])} parameter combinations...')
grid.fit(X_train, y_train)

print('\n✅ GridSearch completed!')
print(f'Best AUC (cv, weighted ovr): {grid.best_score_:.4f}')
print('Best params:')
for k, v in grid.best_params_.items():
    print(f'  {k}: {v}')

best_rf = grid.best_estimator_

Starting GridSearchCV with 324 parameter combinations...
Fitting 5 folds for each of 324 candidates, totalling 1620 fits


/opt/anaconda3/lib/python3.13/site-packages/sklearn/model_selection/_validation.py:960: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/site-packages/sklearn/model_selection/_validation.py", line 949, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "/opt/anaconda3/lib/python3.13/site-packages/sklearn/metrics/_scorer.py", line 288, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.13/site-packages/sklearn/metrics/_scorer.py", line 388, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.13/site-packages/sklear

[CV] END bootstrap=True, class_weight=None, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.3s
[CV] END bootstrap=True, class_weight=None, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.4s
[CV] END bootstrap=True, class_weight=None, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.4s


/opt/anaconda3/lib/python3.13/site-packages/sklearn/model_selection/_validation.py:960: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/site-packages/sklearn/model_selection/_validation.py", line 949, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "/opt/anaconda3/lib/python3.13/site-packages/sklearn/metrics/_scorer.py", line 288, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.13/site-packages/sklearn/metrics/_scorer.py", line 388, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.13/site-packages/sklear

[CV] END bootstrap=True, class_weight=None, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.5s
[CV] END bootstrap=True, class_weight=None, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.5s


...

[CV] END bootstrap=True, class_weight=None, max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   1.3s
[CV] END bootstrap=True, class_weight=None, max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   1.3s
[CV] END bootstrap=True, class_weight=None, max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   1.4s
[CV] END bootstrap=True, class_weight=None, max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.4s
[CV] END bootstrap=True, class_weight=None, max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.5s


/opt/anaconda3/lib/python3.13/site-packages/sklearn/model_selection/_validation.py:960: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/site-packages/sklearn/model_selection/_validation.py", line 949, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "/opt/anaconda3/lib/python3.13/site-packages/sklearn/metrics/_scorer.py", line 288, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.13/site-packages/sklearn/metrics/_scorer.py", line 388, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.13/site-packages/sklear

[CV] END bootstrap=True, class_weight=None, max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.5s
[CV] END bootstrap=True, class_weight=None, max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   1.6s
[CV] END bootstrap=True, class_weight=None, max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.3s


/opt/anaconda3/lib/python3.13/site-packages/sklearn/model_selection/_validation.py:960: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/site-packages/sklearn/model_selection/_validation.py", line 949, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "/opt/anaconda3/lib/python3.13/site-packages/sklearn/metrics/_scorer.py", line 288, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.13/site-packages/sklearn/metrics/_scorer.py", line 388, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.13/site-packages/sklear

[CV] END bootstrap=True, class_weight=None, max_depth=None, max_features=log2, min_samples_leaf=4, min_samples_split=10, n_estimators=500; total time=   2.2s


In [ ]:
# Evaluation: metrics, confusion matrix, ROC/AUC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Predict on test
y_pred = best_rf.predict(X_test)
y_proba = best_rf.predict_proba(X_test)

# Accuracy and classification report
acc = accuracy_score(y_test, y_pred)
print(f'Accuracy: {acc:.4f}')
print('\nClassification report:')
print(classification_report(y_test, y_pred, digits=4))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Away','Draw','Home'],
            yticklabels=['Away','Draw','Home'])
plt.title('Confusion Matrix - RandomForest')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.tight_layout()
plt.show()

# ROC-AUC (multiclass, OVR) and ROC curves per class
classes = np.unique(y)
y_test_bin = label_binarize(y_test, classes=classes)  # shape (n_samples, n_classes)

macro_auc = roc_auc_score(y_test, y_proba, multi_class='ovr', average='macro')
weighted_auc = roc_auc_score(y_test, y_proba, multi_class='ovr', average='weighted')
print(f'ROC-AUC (macro OVR):    {macro_auc:.4f}')
print(f'ROC-AUC (weighted OVR): {weighted_auc:.4f}')

# Plot ROC curves per class
plt.figure(figsize=(7,6))
for i, cls in enumerate(classes):
    from sklearn.metrics import roc_curve, auc
    fpr, tpr, _ = roc_curve(y_test_bin[:, i], y_proba[:, i])
    roc_auc = auc(fpr, tpr)
    label = {0:'Away', 1:'Draw', 2:'Home'}.get(int(cls), str(cls)) + f' (AUC={roc_auc:.3f})'
    plt.plot(fpr, tpr, lw=2, label=label)

plt.plot([0,1],[0,1], color='gray', lw=1, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves by Class - RandomForest (OVR)')
plt.legend(loc='lower right')
plt.tight_layout()
plt.show()

In [ ]:
# Feature importance and CV results summary
import pandas as pd
import numpy as np
importances = pd.Series(best_rf.feature_importances_, index=X_train.columns).sort_values(ascending=False)
plt.figure(figsize=(8,6))
sns.barplot(x=importances.values[:15], y=importances.index[:15], color='#1f77b4')
plt.title('Top Feature Importances - RandomForest')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()

# Top CV configs by AUC
cv_results = pd.DataFrame(grid.cv_results_)
cols = ['mean_test_score','std_test_score','mean_train_score','std_train_score','params']
display(cv_results.sort_values('mean_test_score', ascending=False)[cols].head(10))